In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import joblib

# Step 1: Load the data
data = pd.read_csv('labelled_train_set.csv')

# Step 2: Preprocess the data
# Drop any rows with missing values
data.dropna(inplace=True)

# Convert text to lowercase
data['News/Comment'] = data['News/Comment'].str.lower()

# Encode the target variable
data['Type'] = data['Type'].map({'FALSE': 0, 'HALF TRUE': 1, 'TRUE': 2})

# Step 3: Split the data into features and target
X = data['News/Comment']
y = data['Type']

# Handle Missing Values in Target Variable
y = y.fillna(y.mode()[0]) # Fill missing values with the most frequent class

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 5: Vectorize the text data
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Step 6: Handle class imbalance using SMOTE
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

# Step 7: Train the model
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train_resampled, y_train_resampled)

# Step 8: Evaluate the model
y_pred = rf_model.predict(X_test_vectorized)

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Optional: Save the model and vectorizer for future use
joblib.dump(rf_model, 'fake_news_model.pkl')
joblib.dump(vectorizer, 'count_vectorizer.pkl')

# ---------------------------------------
# For making predictions on new test data
# ---------------------------------------

import pandas as pd
import joblib

# Step 1: Load the saved model and vectorizer
model = joblib.load('fake_news_model.pkl')
vectorizer = joblib.load('count_vectorizer.pkl')

# Step 2: Load the new test data
test_data = pd.read_csv('unlabelled_test1.csv')  # Replace with your test data file path

# Ensure that the test data contains an 'ID' column
if 'ID' not in test_data.columns:
    raise ValueError("Test data must contain an 'ID' column.")

# Step 3: Preprocess the test data
# Convert text to lowercase
test_data['News/Comment'] = test_data['News/Comment'].str.lower()

# Step 4: Vectorize the test data
X_test_vectorized = vectorizer.transform(test_data['News/Comment'])

# Step 5: Make predictions
predictions = model.predict(X_test_vectorized)

# Step 6: Add predictions to the test data
test_data['Predicted_Type'] = predictions

# Step 7: Save the results with 'ID' and 'Predicted_Type'
test_data[['ID', 'Predicted_Type']].to_csv('predicted_test_results.csv', index=False)

print("Predictions saved to 'predicted_test_results.csv'")


              precision    recall  f1-score   support

         0.0       0.90      0.81      0.85       279
         1.0       0.05      0.11      0.07        28

    accuracy                           0.74       307
   macro avg       0.48      0.46      0.46       307
weighted avg       0.82      0.74      0.78       307

[[225  54]
 [ 25   3]]
Predictions saved to 'predicted_test_results.csv'
